In [ ]:
from lyricsgenius import Genius
import re
from tokens import *

artists = ['Kim Na Young', 'Heize', 'BOL4', 'OOHYO', 'Damons Year']
cnt = 0
genius = Genius(token)
for a in artists:
    res = genius.search_artist(a)

    for i in range(len(res.__dict__['songs'])):
        lyrics = res.__dict__['songs'][i].__dict__['lyrics']
        title = res.__dict__['songs'][i].title + ' - ' + a
        korean_lyrics = title + '\n' + lyrics
        with open(f'lyrics/{cnt}.txt', 'w') as f:
                f.write(korean_lyrics)
                cnt += 1
        

Searching for songs by Kim Na Young...

Changing artist name to 'Kim Na Young (김나영)'
Song 1: "다시 너를 (Once Again)"
Song 2: "어른이 된다는게 (Being An Adult)"
Song 3: "There for you (English)"
Song 4: "이별후회 (There For You)"
Song 5: "미로 (Maze)"
Song 6: "괜찮다고 (I’m OK)"
Song 7: "가슴이 말해 (Say Goodbye)"
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Kim-na-young-maze-inst-lyrics
Song 8: "미로 (Maze) (Inst.)"
Song 9: "가끔 내가 (Sometimes)"
Song 10: "다른 누구 말고 너야 (Not Anyone Else)"
Song 11: "우연이 아닌것만 같아서 (My All)"
Song 12: "그 한마디 (I Can’t Say That)"
Song 13: "그대만 보여서 (Because I Only See You)"
Song 14: "솔직하게 말해서 나 (To Be Honest)"
Song 15: "숨결 (Breath)"
Song 16: "못해 (I Can’t)"
Song 17: "그대는 어디에 (Where Are You)"
Song 18: "The Youngest Day"
Song 19: "그래 그래 (I See)"
Song 20: "우리 서로 (End of Love)"
Song 21: "그렇게 (Like That)"
Song 22: "너의 모든 기억속에 (Close I’ll Be)"
Song 23: "조금 더 아파도 (Love Hurts A Little More)"
"그 한마디 (I Can’t Say That) [Instrumental]" is not 

In [22]:
import json
# read the dictionary title_ids from title_ids.txt
title_ids = {}
with open('title_ids.json', 'r') as f:
    title_ids = json.load(f)

In [27]:
import re
import os
import regex

folder_path = "lyrics/"
lyrics_dir = "lyrics/"
i = 0
missing_ids = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and file_path.endswith(".txt"):
        # open the file and read it
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        lines = text.split("\n")
        lyrics = '\n'.join(lines[1:])
        title = lines[0]
        # remove english from the last line
        lyrics = lyrics.split('\n')
        lyrics[0] = regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', lyrics[0])[-1]
        lyrics[-1] = regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', lyrics[-1])[0]
        korean_lyrics = '\n'.join(lyrics)
        korean_lyrics = korean_lyrics.strip().replace('\n\n', '\n')
        
        newlines = []
        for line in lyrics:
            if re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', line):
                if not re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[-1]):
                    newlines.append(regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[-1])
                elif not re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[0]):
                    newlines.append(regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[0])
                else: newlines.append('')
            else:
                newlines.append(line)
                
        korean_lyrics = '\n'.join(line.strip() for line in newlines if not re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', line))
        korean_lyrics = '\n'.join(line.strip() for line in korean_lyrics.split('\n') if len(line.split()) > 1)
        if len(korean_lyrics.split('\n')) > len(lines) * .6: 
            if title in title_ids:
                id = title_ids[title]
            else: id = None
            if id == None:
                missing_ids.append(title)
            json_data = {
                "title": title,
                "lyrics": korean_lyrics.split('\n'),
                "video_id": id
            }

            # Write the JSON object to a JSON file
            json_file_path = os.path.join(lyrics_dir, f"{i}.json")
            with open(json_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(json_data, json_file, ensure_ascii=False, indent=2)
            
            i += 1
print("Missing ids: " + str(missing_ids))
print("Total videos: " + str(i))


Missing ids: []
Total videos: 173


In [24]:
from googleapiclient.discovery import build
import json
import os
from tokens import api_key

youtube = build("youtube", "v3", developerKey=api_key)

# loop through missing ids
for id in missing_ids:
    # Assuming the song title is the first line in the "lyrics" field
    song_title = id

    # Search for videos for the song title
    search_response = youtube.search().list(
        q=song_title,
        type="video",
        part="id",
        maxResults=1
    ).execute()

    if len(search_response['items']) == 0:
        print(f"No videos found for '{song_title}'")
        continue
    else:
        # add the video id to the json file
        title_ids[song_title] = search_response['items'][0]['id']['videoId']

In [7]:
from googleapiclient.discovery import build
import json
import os
from tokens import api_key

youtube = build("youtube", "v3", developerKey=api_key)

folder_path = "lyrics" # Path to the folder containing the JSON files

n_songs = 168
cnt = 0
# Iterate through the files in the specified range
for n in range(0, n_songs):
    file_path = os.path.join(folder_path, f"{n}.json")

    # Check if the file exists
    if os.path.isfile(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # Assuming the song title is the first line in the "lyrics" field
            song_title = data["title"]
            
            # Search for videos for the song title
            search_response = youtube.search().list(
                q=song_title,
                type="video",
                part="id",
                maxResults=1
            ).execute()

            if len(search_response['items']) == 0:
                print(f"No videos found for '{song_title}'")
                continue
            else:
                # add the video id to the json file
                data["video_id"] = search_response['items'][0]['id']['videoId']
                with open(f"lyrics/{cnt}.json", 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)
                    cnt += 1


print("Finished converting videos. Missing ids: " + missing_ids)


KeyboardInterrupt: 

In [26]:
with open('title_ids.json', 'w') as f:
    json.dump(title_ids, f, indent=4)

In [21]:
import json
# loop through lyrics/n.json for n in range(0, 168) and put their video ids in a list
title_ids = {}
for i in range(0, 173):
    with open('lyrics/' + str(i) + '.json', 'r') as f:
        data = json.load(f)
        title = data['title']
        id = data['video_id']
        title_ids[title] = id
#write video_ids to a text file
with open('title_ids.json', 'w') as f:
    json.dump(title_ids, f, indent=4)